In [ ]:
import ecospat.ecospat as ecospat_full
from ecospat.stand_alone_functions import (
    analyze_species_distribution,
    extract_raster_means_single_species,
    analyze_northward_shift,
    process_species_historical_range,
    calculate_rate_of_change_first_last,
    merge_category_dataframes,
    full_propagule_pressure_pipeline,
    prepare_gdf_for_rasterization,
    cat_int_mapping,
    rasterize_multiband_gdf_match,
    rasterize_multiband_gdf_world,
    compute_propagule_pressure_range,
)
from ecospat.references_data import REFERENCES

In [ ]:
map_modern = ecospat_full.Map()
map_modern.add_control_panel()
map_modern.add_basemap_gui()
map_modern

In [ ]:
map_new = ecospat_full.HistoricalMap()
map_new.add_raster(
    "/Users/alivianytko/Downloads/Populus_angustifolia_persistence_raster.tif",
    colormap="viridis",
)
map_new

In [ ]:
from shapely.geometry import Polygon
import geopandas as gpd

# Example polygon — replace this with your actual geometry
polygon = Polygon(
    [
        (-117.78973, 49.08967),
        (-117.78238, 49.08856),
        (-117.783, 49.084),
        (-117.78973, 49.08967),
    ]
)

# Create GeoDataFrame with WGS84 lat/lon CRS
gdf = gpd.GeoDataFrame(geometry=[polygon], crs="EPSG:4326")

# Reproject to a projected CRS suitable for meters (UTM zone for this region: 11N or EPSG:32611)
gdf_meters = gdf.to_crs(epsg=32611)

# Measure the area in square meters
area_m2 = gdf_meters.geometry.area.iloc[0]

# Measure the bounds and diagonal length in meters
bounds = gdf_meters.bounds.iloc[0]
minx, miny, maxx, maxy = bounds
diagonal = ((maxx - minx) ** 2 + (maxy - miny) ** 2) ** 0.5

print(f"Area: {area_m2:.2f} m²")
print(f"Longest diagonal: {diagonal:.2f} meters")

In [ ]:
map_historic = ecospat_full.HistoricalMap()
map_historic.add_control_panel()
map_historic.add_basemap_gui()
map_historic

In [ ]:
map_pop = ecospat_full.HistoricalMap()
map_pop.add_control_panel()
map_pop.add_basemap_gui()
map_pop

In [ ]:
map_test = ecospat_full.HistoricalMap()
map_test.load_historic_data("Populus angustifolia")
map_test

In [ ]:
map = ecospat_full.Map()
hist_range = process_species_historical_range(
    new_map=map, species_name="Populus angustifolia"
)

In [ ]:
classified_modern, classified_historic = analyze_species_distribution(
    "Populus angustifolia", record_limit=2000
)

In [ ]:
northward_rate_df = analyze_northward_shift(
    gdf_hist=hist_range,
    gdf_new=classified_modern,
    species_name="Populus angustifolia",
)
northward_rate_df = northward_rate_df[
    northward_rate_df["category"].isin(["leading", "core", "trailing"])
]

northward_rate_df["category"] = northward_rate_df["category"].str.title()

In [ ]:
print(northward_rate_df)

In [ ]:
change = calculate_rate_of_change_first_last(
    classified_historic, classified_modern, "Populus angustifolia", custom_end_year=2025
)


change = change[change["collapsed_category"].isin(["leading", "core", "trailing"])]
change = change.rename(
    columns={
        "collapsed_category": "Category",
        "rate_of_change_first_last": "Rate of Change",
        "start_time_period": "Start Years",
        "end_time_period": "End Years",
    }
)

# Convert 'Category' column to title case
change["Category"] = change["Category"].str.title()

In [ ]:
full_show, full_save, show_bounds, save_bounds = full_propagule_pressure_pipeline(
    classified_modern, northward_rate_df, change
)

In [ ]:
map_new = ecospat_full.HistoricalMap()
map_new.add_raster(
    "/Users/alivianytko/Downloads/Populus_angustifolia_persistence_raster.tif",
    colormap="viridis",
    legend=True,
)
map_new

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(
    full_show, cmap="viridis", origin="upper"
)  # Choose a colormap ('viridis' is a popular choice)
plt.colorbar(label="Pressure")  # Add colorbar with label
plt.title("Propagule Pressure Map")  # Add a title
plt.xlabel("Longitude")  # Optional: Add x-axis label
plt.ylabel("Latitude")  # Optional: Add y-axis label
plt.show()

In [ ]:
def rasterize_multiband_gdf(gdf, value_columns, out_shape=(50, 50), bounds=None):
    """
    Rasterizes multiple value columns of a GeoDataFrame into a multiband raster.

    Parameters:
    - gdf: GeoDataFrame with polygon geometries and numeric value_columns
    - value_columns: list of column names to rasterize into bands
    - out_shape: shape of the raster (height, width)
    - bounds: bounding box (minx, miny, maxx, maxy). If None, computed from gdf.

    Returns:
    - 3D numpy array (bands, height, width)
    - affine transform
    """
    import numpy as np
    import rasterio
    from rasterio.features import rasterize
    from rasterio.transform import from_bounds

    # Calculate bounds if not given
    if bounds is None:
        bounds = gdf.total_bounds  # (minx, miny, maxx, maxy)

    minx, miny, maxx, maxy = bounds
    height, width = out_shape
    transform = from_bounds(minx, miny, maxx, maxy, width, height)

    bands = []

    for col in value_columns:
        shapes = [(geom, value) for geom, value in zip(gdf.geometry, gdf[col])]
        raster = rasterize(
            shapes,
            out_shape=out_shape,
            transform=transform,
            fill=np.nan,
            dtype="float32",
        )
        bands.append(raster)

    stacked = np.stack(bands, axis=0)  # shape: (bands, height, width)
    return stacked, transform, bounds